# RASA x LanceDB x LLM : Conversational chatbot

Rasa is an open-source framework for building intelligent, contextual, and scalable conversational agents. It provides robust tools for **natural language understanding (NLU)**,**dialogue management** and **custom actions**, enabling the creation of sophisticated chatbots and virtual assistants tailored to specific business needs. With its flexible architecture, Rasa allows seamless integration with various APIs, databases, and machine learning models, facilitating the development of highly responsive and intelligent conversational experiences.

Explore RASA at : https://rasa.com/docs/rasa/

Explore LanceDB at : https://lancedb.github.io/lancedb/

![](https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExenZjZDhsbnp2cGxibW1zbmQ1Ymc2ZTgzbzBzbDRwZ2t3MzZsd3hvaCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/S0hxMGYFhEMzm/giphy.webp)

Today we will be building an **Advanced Customer Support Chatbot** using Rasa, LanceDB, and OpenAI api to deliver a great customer service experience. This chatbot is designed to handle a wide range of customer inquiries, provide accurate information, and offer personalized assistance, all while maintaining a natural and engaging conversational flow.






In [ ]:
# Install required packages
!pip install rasa lancedb openai==0.28 python-dotenv -q

In [ ]:
#Initialize a new Rasa project which sets up the necessary directory structure and files.
!rasa init --no-prompt

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
(0lqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqk(B
(0x(B Rasa Open Source reports anonymous usage telemetry to help improve the product (0x(B
(0x(B for all its users.                                                             (0x(B
(0x(B                                                                                (0x(B
(0x(B If you'd like to opt-out, you can use `ra

# Quick overview of RASA x LanceDB Integration Workflow

**Step 1 : Define knowledge_data and store it in LanceDB:**
* Use LanceDB to store and manage structured knowledge data relevant to customer support.

**Step 2 : Configure Rasa Files:**
- nlu.yml: Train intent recognition and entity extraction.
- stories.yml & rules.yml: Define conversational flows and rules.
- domain.yml: Specify intents, entities, actions, and responses.
- config.yml: Set up the NLP pipeline and policies.

**Step 3 : Implement Custom Actions (actions.py):**
- Create actions that query LanceDB for relevant information based on user intents.
- Integrate OpenAI’s LLM to generate nuanced and context-aware responses.

**Step 4 : Train the Model (rasa train):**
- Compile and optimize the Rasa model based on the latest configurations and training data.

**Step 5 : Run Servers:**
- Action Server: Executes custom actions.
- Rasa Server: Handles user interactions and manages dialogue using the trained model.

**Step 6 : Deploy and Test:**
- Interact with the chatbot to ensure it accurately understands queries, retrieves information from LanceDB, and generates appropriate responses via OpenAI’s LLM.

### Creating and storing the Knowledge base of Company Support in a Lance Table

Let's create a dataset containing customer support information. We will use this data to populate LanceDB table.

It contains a list of dictionaries where each dictionary represents a piece of knowledge or an FAQ entry. Each entry has a "content" key corresponding to the support information.

In [ ]:
# This dataset covers various categories such as Account Management, Billing & Payments, Technical Support, Shipping & Delivery, Returns & Refunds, Product Information, and Company Policies.

company_support_data = [
    # === Account Management ===
    {
        "content": "To reset your password, navigate to the login page and click on 'Forgot Password'. You'll receive an email with instructions to create a new password."
    },
    {
        "content": "To update your account information, log in to your profile and click on 'Edit Profile'. From there, you can change your email, phone number, and other personal details."
    },
    {
        "content": "If you want to delete your account, please contact our support team at support@yourcompany.com with the subject line 'Account Deletion Request'. We'll assist you promptly."
    },
    {
        "content": "To change your username, go to your account settings, select 'Username', and enter your desired new username. If it's available, the change will be applied immediately."
    },

    # === Billing & Payments ===
    {
        "content": "We accept various payment methods including Visa, MasterCard, American Express, PayPal, and Apple Pay."
    },
    {
        "content": "To view your billing history, log in to your account, navigate to the 'Billing' section, and select 'Billing History'. You'll see a list of all your past transactions."
    },
    {
        "content": "If you believe you've been incorrectly charged, please contact our billing support at billing@yourcompany.com with your order ID and details of the discrepancy."
    },
    {
        "content": "Your subscription will automatically renew on the renewal date unless you cancel it at least 24 hours before the renewal date."
    },

    # === Technical Support ===
    {
        "content": "If you encounter issues while using our platform, try clearing your browser cache and cookies, then restart your browser."
    },
    {
        "content": "For mobile app issues, ensure that you have the latest version of the app installed. You can update the app from the App Store or Google Play Store."
    },
    {
        "content": "To report a bug, please email us at support@yourcompany.com with a detailed description of the issue and any relevant screenshots."
    },
    {
        "content": "Our platform requires JavaScript to be enabled. Please check your browser settings to ensure JavaScript is turned on."
    },

    # === Shipping & Delivery ===
    {
        "content": "We offer free standard shipping on all orders over $50. Standard shipping typically takes 5-7 business days."
    },
    {
        "content": "Express shipping options are available at an additional cost. Delivery usually takes 1-3 business days."
    },
    {
        "content": "You can track your order status by logging into your account and visiting the 'Orders' section. Click on your order to see detailed tracking information."
    },
    {
        "content": "If your package is delayed, please contact our shipping department at shipping@yourcompany.com with your order number for assistance."
    },

    # === Returns & Refunds ===
    {
        "content": "You can return most items within 30 days of purchase. Items must be in original condition and packaging."
    },
    {
        "content": "To initiate a return, log in to your account, go to the 'Orders' section, select the order you wish to return, and click on 'Start Return'. Follow the on-screen instructions."
    },
    {
        "content": "Refunds are processed within 7-10 business days after we receive your returned item. The refund will be issued to your original payment method."
    },
    {
        "content": "Please note that shipping costs are non-refundable unless the return is due to a defect or error on our part."
    },

    # === Order Management ===
    {
        "content": "To modify your order after placing it, please contact our support team at support@yourcompany.com as soon as possible. Changes can only be made within one hour of order placement."
    },
    {
        "content": "If you received the wrong item, please contact us immediately with your order number and a photo of the received product. We'll arrange for a replacement or refund."
    },
    {
        "content": "To cancel your order, log in to your account, go to the 'Orders' section, select the order you wish to cancel, and click on 'Cancel Order'."
    },
    {
        "content": "Parts of your order may ship separately. You will receive separate tracking information for each shipment."
    },

    # === Account Security ===
    {
        "content": "For enhanced security, enable two-factor authentication (2FA) in your account settings. This adds an extra layer of protection to your account."
    },
    {
        "content": "If you suspect unauthorized access to your account, change your password immediately and contact our support team."
    },
    {
        "content": "Never share your account credentials with anyone. We will never ask for your password via email or phone."
    },
    {
        "content": "Regularly review your account activity to ensure all actions are authorized. You can view your recent activity in the 'Account Activity' section."
    },

    # === Shipping Issues ===
    {
        "content": "If your package hasn't arrived within the estimated delivery time, please use the tracking number in your order details to check its status, or contact our shipping support at shipping@yourcompany.com."
    },
    {
        "content": "For international shipments, please note that customs processing times may affect delivery dates. We are not responsible for delays caused by customs."
    },
    {
        "content": "Ensure that your shipping address is correct before placing an order. Incorrect addresses may lead to delivery delays or package returns."
    },
    {
        "content": "We offer shipment insurance for an additional fee. This covers lost or damaged packages during transit. To add insurance, select the option during checkout."
    },

    # === Product Availability ===
    {
        "content": "If a product is out of stock, you can sign up for restock notifications on the product page. We'll notify you via email once it's available again."
    },
    {
        "content": "Limited edition items are subject to availability. We recommend placing your orders early to secure these exclusive products."
    },
    {
        "content": "Pre-orders are accepted for upcoming releases. Pre-order now to ensure you receive the product as soon as it ships."
    },
    {
        "content": "Sold-out items cannot be purchased until new stock arrives. Please check back regularly or join our waitlist."
    },

    # === Shipping Costs ===
    {
        "content": "Shipping costs vary based on your location and the size of your order. You can view the estimated shipping cost during the checkout process."
    },
    {
        "content": "We offer free standard shipping on orders over $75. Orders below this threshold will incur standard shipping fees."
    },
    {
        "content": "Express shipping is available for an additional fee. If you need your order faster, select the express option at checkout."
    },
    {
        "content": "Please note that international shipping fees will apply for orders shipped outside the United States."
    },

    # === Subscription Services ===
    {
        "content": "Subscribe to our monthly box to receive exclusive products and discounts. You can manage your subscription in your account settings."
    },
    {
        "content": "Canceling your subscription can be done at any time. Please visit the 'Subscriptions' section in your account to make changes."
    },
    {
        "content": "Subscribers receive early access to new products and special promotions throughout the year."
    },
    {
        "content": "Pause your subscription if you need a break. You can resume it whenever you're ready by logging into your account."
    },

    # === Order Tracking ===
    {
        "content": "To track your order, log in to your account, go to the 'Orders' section, and click on 'Track Order' next to the relevant order."
    },
    {
        "content": "You'll receive email notifications with tracking updates once your order has been shipped."
    },
    {
        "content": "If you haven't received your tracking information, please check your spam folder or contact our support team."
    },
    {
        "content": "For real-time tracking, use the tracking number provided and visit the carrier's website directly."
    },

    # === Account Reactivation ===
    {
        "content": "If you've previously closed your account and wish to reactivate it, please contact our support team at support@yourcompany.com with your account details."
    },
    {
        "content": "Reactivating your account will restore your previous order history and saved preferences."
    },
    {
        "content": "Please note that some data may have been permanently deleted if the account was closed more than six months ago."
    },
    {
        "content": "Upon reactivation, you may need to update your password and verify your email address for security purposes."
    },

    # === Payment Issues ===
    {
        "content": "If your payment is declined, please verify that your billing information is correct and that your card has sufficient funds."
    },
    {
        "content": "For any issues with payments made through PayPal, please ensure your account is in good standing and that your linked bank accounts are active."
    },
    {
        "content": "Expired or invalid credit cards cannot be used. Please update your payment method in your account settings."
    },
    {
        "content": "If you've been double-charged, contact our billing support at billing@yourcompany.com with your order ID and transaction details."
    },

    # === Warranty Information ===
    {
        "content": "All our products come with a one-year warranty covering manufacturing defects and hardware malfunctions."
    },
    {
        "content": "To claim a warranty, contact our support team with your order number and a description of the issue."
    },
    {
        "content": "Warranty does not cover accidental damage, unauthorized repairs, or normal wear and tear."
    },
    {
        "content": "Extended warranty options are available at the time of purchase for an additional fee."
    },

    # === Privacy & Data Security ===
    {
        "content": "We prioritize your privacy and ensure that your personal data is protected. Please refer to our Privacy Policy for detailed information."
    },
    {
        "content": "You have control over your personal information. Manage your privacy settings in the 'Account Settings' section."
    },
    {
        "content": "We do not share your personal information with third parties without your explicit consent, except as required by law."
    },
    {
        "content": "For any concerns about data security or privacy, contact our privacy team at privacy@yourcompany.com."
    },

    # === Shipping Restrictions ===
    {
        "content": "We currently do not ship to P.O. Boxes or APO/FPO addresses. Please provide a valid residential or business address."
    },
    {
        "content": "Certain products may have shipping restrictions based on international regulations. Check the product page for any specific restrictions."
    },
    {
        "content": "Hazardous materials and perishable items have special shipping requirements. Ensure compliance by reviewing our shipping guidelines."
    },
    {
        "content": "Import duties and taxes are the responsibility of the recipient for international orders. These fees are not included in the shipping cost."
    },

    # === Frequently Asked Questions (FAQs) ===
    {
        "content": "Q: How can I track my order?\nA: You can track your order by logging into your account, navigating to the 'Orders' section, and clicking on 'Track Order' next to your order."
    },
    {
        "content": "Q: What is your return policy?\nA: We accept returns within 30 days of purchase. Items must be in original condition and packaging. To initiate a return, visit the 'Orders' section in your account."
    },
    {
        "content": "Q: Do you offer international shipping?\nA: Yes, we ship to select countries internationally. Shipping fees and delivery times may vary based on location."
    },
    {
        "content": "Q: How do I change my shipping address?\nA: To change your shipping address, go to your account settings, click on 'Edit Address', and update your details. Note that changes can only be made before the order is shipped."
    },
    {
        "content": "Q: Can I cancel my order?\nA: Yes, you can cancel your order within one hour of placing it. To cancel, go to the 'Orders' section in your account and select 'Cancel Order'."
    },
    {
        "content": "Q: How secure is my personal information?\nA: We use industry-standard encryption and security measures to protect your personal information. Refer to our Privacy Policy for more details."
    },
    {
        "content": "Q: What payment methods are accepted?\nA: We accept Visa, MasterCard, American Express, PayPal, Apple Pay, and Google Pay."
    },
    {
        "content": "Q: How do I apply a promo code?\nA: During checkout, enter your promo code in the 'Promo Code' field and click 'Apply' to receive your discount."
    },
    {
        "content": "Q: Who do I contact for support?\nA: For any inquiries or assistance, contact our support team at support@yourcompany.com or call us at (123) 456-7890."
    },

    # === Account Verification ===
    {
        "content": "To verify your account, check your email for a verification link after signing up. Click on the link to activate your account."
    },
    {
        "content": "If you did not receive a verification email, please check your spam folder or resend the verification email from your account settings."
    },
    {
        "content": "Account verification is required to access all features of our platform, including order placement and account management."
    },
    {
        "content": "For any issues with account verification, contact our support team at support@yourcompany.com."
    },

    # === Contact Information ===
    {
        "content": "You can reach our support team via email at support@yourcompany.com or call us at (123) 456-7890 from Monday to Friday, 9 AM to 5 PM EST."
    },
    {
        "content": "Follow us on social media for the latest updates, promotions, and customer support: Facebook, Twitter, Instagram."
    },
    {
        "content": "Visit our 'Contact Us' page on our website for a direct contact form and additional support options."
    },
    {
        "content": "Our headquarters are located at 1234 Business Ave, Suite 100, City, State, ZIP Code."
    },

    # === Order Confirmation ===
    {
        "content": "After placing an order, you'll receive a confirmation email with your order details and estimated delivery date."
    },
    {
        "content": "If you haven't received your order confirmation, please check your spam folder or contact our support team for assistance."
    },
    {
        "content": "The order confirmation email includes a summary of your purchase, billing information, and contact details for support."
    },
    {
        "content": "For large orders or bulk purchases, you'll receive a separate confirmation email with additional details."
    },

    # === Shipping Options ===
    {
        "content": "We offer three shipping options: Standard, Express, and Overnight. Choose the one that best fits your needs during checkout."
    },
    {
        "content": "Standard shipping is free for orders over $75. It typically takes 5-7 business days for delivery."
    },
    {
        "content": "Express shipping costs are calculated based on the weight and destination of your order. Estimated delivery is 2-3 business days."
    },
    {
        "content": "Overnight shipping is available for urgent orders. Placing your order before 10 AM ensures next-day delivery."
    },

    # === Custom Order Handling ===
    {
        "content": "For custom orders, please contact our support team at support@yourcompany.com with detailed specifications and requirements."
    },
    {
        "content": "Custom orders may require additional processing time. Delivery dates will be provided after discussing your specific needs."
    },
    {
        "content": "Pricing for custom orders varies based on the complexity and materials involved. Our support team will provide a quote upon request."
    },
    {
        "content": "Please review all custom order details carefully before finalizing to ensure accuracy and satisfaction."
    },
    # === FAQs Specific ===
    {
        "content": "Q: How do I track my order?\nA: You can track your order by logging into your account, going to the 'Orders' section, and clicking on 'Track Order' next to your recent purchase."
    },
    {
        "content": "Q: What is your return policy?\nA: We accept returns within 30 days of purchase. Items must be in their original condition and packaging. To initiate a return, visit the 'Orders' section in your account and select 'Return Item'."
    },
    {
        "content": "Q: Do you offer international shipping?\nA: Yes, we ship to select countries internationally. Shipping fees and delivery times vary based on your location. Please refer to the 'Shipping Information' page for more details."
    },
    {
        "content": "Q: How can I contact customer support?\nA: You can contact our support team via email at support@yourcompany.com or call us at (123) 456-7890 during our support hours, Monday to Friday, 9 AM to 5 PM EST."
    },
    {
        "content": "Q: What payment methods do you accept?\nA: We accept Visa, MasterCard, American Express, PayPal, Apple Pay, and Google Pay."
    },
    # ... Add more FAQs as needed
]


Let's set up a robust knowledge base for an advanced customer support chatbot by leveraging LanceDB for efficient data storage and retrieval, and a Sentence Transformer model for generating vector embeddings.

We will be using pre-trained Sentence Transformer model (BAAI/bge-small-en-v1.5) to convert textual support data into vector embeddings. You can change it accordingly.

Then we will create a table - "knowledge_base" in the DB and insert all the customer support information in this table.

In [ ]:
# Import necessary libraries
import os
import subprocess
import time
import threading
import lancedb
import requests
import json
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

In [ ]:
# Initialize LanceDB
db = lancedb.connect("./content/lancedb")  # Local storage within Colab

# Initialize the language model for generating embeddings
model = get_registry().get("sentence-transformers").create(name="BAAI/bge-small-en-v1.5", device="cpu")

# Create table from schema
class Documents(LanceModel):
    vector: Vector(model.ndims()) = model.VectorField()
    content: str = model.SourceField()          # Field to store the actual content/response


# Knowledge data from the data above
knowledge_data = company_support_data

# Define table name
table_name = "knowledge_base"

# Retrieve existing table names
existing_tables = db.table_names()

if table_name not in existing_tables:
    # Create a new table with the schema and insert data
    tbl = db.create_table(table_name, schema=Documents)
    tbl.add(knowledge_data)
    print(f"Created new table '{table_name}' and inserted data.")
else:
    # Append data to the existing table
    table = db.open_table(table_name)
    table.add(knowledge_data, mode="overwrite")
    print(f"Overwrited data to the existing table '{table_name}'.")


Overwrited data to the existing table 'knowledge_base'.


###  Configure Rasa Files

#### domain.yml
The domain.yml file serves as the core configuration for your Rasa chatbot. It defines the chatbot’s intents, entities, slots, responses, actions, forms, and policies.

It has the following Components -
1. **Intents**: Represents the purposes behind user messages (e.g., greetings, inquiries about account deletion).
2. **Entities**: Specific pieces of information extracted from user inputs (e.g., project names, dates).
3. **Responses**: Predefined messages the chatbot can send back to users.
4. **Actions**: Lists both built-in and custom actions that the chatbot can perform.
5. **Slots**: Variables that store information during a conversation to maintain context and continuity.



In [ ]:
%%writefile domain.yml
version: "3.0"

language: "en"

intents:
  - greet
  - ask_knowledge
  - goodbye

entities:
  - project
  - service

responses:
  utter_greet:
    - text: "Hello! How can I assist you today?"

  utter_goodbye:
    - text: "Goodbye! Have a great day!"
    - text: "Bye! Let me know if you need anything else."
    - text: "See you later! Feel free to reach out anytime."

actions:
  - action_search_knowledge


Overwriting domain.yml


#### endpoints.yml
The endpoints.yml file configures various backend services that Rasa interacts with, such as action servers, event brokers, and tracker stores. It specifies the URLs and connection details for services like the custom action server (actions.py), enabling Rasa to communicate with it.

Following configurations are relevant to this use-case -
1. **Action Endpoint**: Defines where Rasa should send requests to execute custom actions, typically pointing to the server running actions.py.
2. **Tracker Store**: (Optional) Configures how conversation states are stored, which can be essential for maintaining session information in more complex setups.

In [ ]:
%%writefile endpoints.yml
# This file contains the different endpoints your bot can use.

# Server where the models are pulled from.
# https://rasa.com/docs/rasa/model-storage#fetching-models-from-a-server

#models:
#  url: http://my-server.com/models/default_core@latest
#  wait_time_between_pulls:  10   # [optional](default: 100)

# Server which runs your custom actions.
# https://rasa.com/docs/rasa/custom-actions

action_endpoint:
  url: "http://localhost:5055/webhook"

# Tracker store which is used to store the conversations.
# By default the conversations are stored in memory.
# https://rasa.com/docs/rasa/tracker-stores

#tracker_store:
#    type: redis
#    url: <host of the redis instance, e.g. localhost>
#    port: <port of your redis instance, usually 6379>
#    db: <number of your database within redis, e.g. 0>
#    password: <password used for authentication>
#    use_ssl: <whether or not the communication is encrypted, default false>

#tracker_store:
#    type: mongod
#    url: <url to your mongo instance, e.g. mongodb://localhost:27017>
#    db: <name of the db within your mongo instance, e.g. rasa>
#    username: <username used for authentication>
#    password: <password used for authentication>

# Event broker which all conversation events should be streamed to.
# https://rasa.com/docs/rasa/event-brokers

#event_broker:
#  url: localhost
#  username: username
#  password: password
#  queue: queue


Overwriting endpoints.yml


#### stories.yml

The stories.yml file contains training stories that define example conversational paths your chatbot can take. These stories help Rasa understand how to manage dialogues by learning from example interactions. It provides Rasa with sequences of user intents and corresponding actions, teaching it how to handle various conversational scenarios.


In [ ]:
%%writefile data/stories.yml
version: "3.0"

stories:
  - story: Greet and ask question
    steps:
      - intent: greet
      - action: utter_greet
      - intent: ask_knowledge
      - action: action_search_knowledge

  - story: ask question
    steps:
      - intent: ask_knowledge
      - action: action_search_knowledge

  - story: Goodbye
    steps:
      - intent: goodbye
      - action: utter_goodbye

  - story: greet and goodbye
    steps:
      - intent: greet
      - action: utter_greet
      - intent: goodbye
      - action: utter_goodbye

Overwriting data/stories.yml


#### rules.yml
The rules.yml file defines rule-based conversations that specify exact steps the chatbot should follow in certain situations. Unlike stories, rules are strict paths that Rasa should follow without deviation. It ensures that in specific scenarios, the chatbot responds in a predefined manner, which is crucial for critical interactions like error handling or confirmations.

It works alongside stories to cover fixed conversational flows that require precise control, such as security checks or policy confirmations.

In [ ]:
%%writefile data/rules.yml
version: "3.0"

rules:
  - rule: Greet
    steps:
      - intent: greet
      - action: utter_greet

  - rule: Goodbye
    steps:
      - intent: goodbye
      - action: utter_goodbye

  - rule: Answer Knowledge Questions
    steps:
      - intent: ask_knowledge
      - action: action_search_knowledge


Overwriting data/rules.yml


#### nlu.yml
The nlu.yml file contains Natural Language Understanding (NLU) training data. It includes examples of user inputs categorized by intents and annotated with entities to train Rasa’s NLU component.

It has the following role in the project -
1. **Intent Recognition**: Enables Rasa to accurately identify the user’s intent from their message.
2. **Entity Extraction**: Allows Rasa to extract specific information from user inputs, which can be used in custom actions to query LanceDB or generate responses via OpenAI’s LLM.
3. **Training Data Quality**: The quality and diversity of examples in nlu.yml directly impact the chatbot’s ability to understand and respond appropriately to a wide range of user queries.

In [ ]:
%%writefile data/nlu.yml
# data/nlu.yml

version: "3.0"

nlu:
  - intent: greet
    examples: |
      - hello
      - hi
      - hey
      - good morning
      - good evening
      - greetings

  - intent: goodbye
    examples: |
      - bye
      - goodbye
      - see you later
      - catch you later
      - see ya
      - take care

  - intent: ask_knowledge
    examples: |
      - I need help with my account
      - Can you assist me with billing?
      - How do I reset my password?
      - I'm facing issues with my order
      - Tell me about your support services
      - How can I contact customer service?
      - What are your support hours?
      - I have a question about Project Alpha
      - Help me understand Project Beta
      - How can I track my purchase?


Overwriting data/nlu.yml


#### config.yml

The config.yml file defines the pipeline and policies used by Rasa for processing natural language inputs and managing dialogue workflows.

Let us understand the components of the config.yml file
1. **NLP Pipeline**: Defines the components that handle preprocessing, feature extraction, intent classification, and entity recognition.
2. **Policies**: Dictate how the chatbot selects actions based on the current dialogue state, using rules, machine learning models, or a combination of both.
3. **Endpoint Integrations**: Can include configurations for connecting to external services or APIs, enhancing the chatbot’s capabilities.

It specifies the sequence of components (like tokenizers, featurizers, classifiers) that process user inputs for intent recognition and entity extraction. Then it determines how Rasa decides the next action based on the conversation state and the trained model.

In [ ]:
%%writefile config.yml
# config.yml
version: "3.0"

language: "en"

pipeline:
- name: WhitespaceTokenizer
- name: RegexFeaturizer
- name: LexicalSyntacticFeaturizer
- name: CountVectorsFeaturizer
- name: CountVectorsFeaturizer
  analyzer: char_wb
  min_ngram: 1
  max_ngram: 4
- name: DIETClassifier
  epochs: 100
- name: EntitySynonymMapper
- name: ResponseSelector
  epochs: 100

policies:
- name: RulePolicy
- name: UnexpecTEDIntentPolicy
  max_history: 5
  epochs: 100
- name: TEDPolicy
  max_history: 5
  epochs: 100
assistant_id: 20241227-151505-young-attachment

Overwriting config.yml


### Implement Custom Actions (actions.py) file

The actions.py file is where you define custom actions for your Rasa chatbot. Custom actions are Python functions that can execute arbitrary logic, such as querying a database, calling external APIs, processing data, or integrating with services like LanceDB and OpenAI’s LLMs.

We will be doing the following in this file -
1. **Get user input**: Processes user inputs and proceed to next step.
2. **Interacting with Databases**: Queries LanceDB to retrieve relevant information based on user queries.
3.**Enhancing Responses with LLMs**: Utilizes OpenAI’s API to generate detailed, contextually relevant responses that go beyond predefined replies.

In [ ]:
%%writefile actions/actions.py
from typing import Any, Text, Dict, List
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
import lancedb
import logging
from google.colab import userdata
import openai
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Configure logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

class ActionSearchKnowledge(Action):
    def name(self) -> Text:
        return "action_search_knowledge"

    def __init__(self):

        # Initialize OpenAI API key from environment variables
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if not self.openai_api_key:
            logger.error("OpenAI API key not found. Please set OPENAI_API_KEY in your environment.")
        openai.api_key = self.openai_api_key

        # Initialize LanceDB connection once
        try:
            self.db = lancedb.connect("./content/lancedb")
            self.table_name = "knowledge_base"
            if self.table_name not in self.db.table_names():
                logger.error(f"Table '{self.table_name}' does not exist in LanceDB.")
                self.table = None
            else:
                self.table = self.db.open_table(self.table_name)
                logger.info(f"Connected to table '{self.table_name}' in LanceDB.")
        except Exception as e:
            logger.error(f"Error connecting to LanceDB: {e}")
            self.table = None

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:


        # Get the latest user message
        user_message = tracker.latest_message.get('text')
        logger.info(f"User message: {user_message}")

        if not user_message:
            dispatcher.utter_message(text="Sorry, I didn't catch that. Could you please repeat?")
            return []

        try:
            # Perform similarity search in LanceDB
            query_result = self.table.search(user_message).limit(1).to_pandas()

            # Filter results based on the _distance parameter (smaller _distance means more similar)
            relevant_content = [query_result.loc[0, "content"] if query_result.loc[0, "_distance"] < 0.65 else None][0]
            response_text = "Null"

            # If we find relevant content , sent it to LLM or Else send automated reply
            if not relevant_content == None:
                logger.info(f"Retrieved answer from knowledge base.")
                # Use OpenAI to generate a more refined response
                response_text = self.generate_response(user_message, relevant_content)
            else:
                # If user has ask not a relevant question, reply with the following
                response_text = "I'm sorry, I don't have an answer to that question."
                logger.info(f"No matching content found in knowledge base.")

            # Send the answer back to the user
            dispatcher.utter_message(text=response_text)

        except Exception as e:
            logger.error(f"Error during search operation: {e}")
            dispatcher.utter_message(text="Sorry, something went wrong while processing your request.")

        return []

    def generate_response(self, user_message: Text, relevant_content: Text) -> Text:
        """
        Use OpenAI's API to generate a refined response based on user message and relevant content.
        """
        try:
            system_prompt = "You are an company support assistant that provides helpful and accurate answers based on the provided information. You talk professionally and like a customer support executive."

            prompt = (
                f"User Question: {user_message}\n"
                f"Relevant Information: {relevant_content}\n\n"
                f"Provide a detailed and helpful response to the user's question based on the relevant information above."
            )

            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=450,
                temperature=0.7,
            )

            generated_text = response.choices[0].message['content'].strip()
            logger.info("Generated response using OpenAI API.")
            return generated_text

        except Exception as e:
            logger.error(f"Error generating response with OpenAI API: {e}")
            return relevant_content  # Fallback to relevant content if OpenAI fails

Overwriting actions/actions.py


### Training RASA Model
The rasa train command is used to train the Rasa model based on the data and configurations defined in your project files (nlu.yml, stories.yml, rules.yml, etc.)

It compiles the training data into a machine learning model that Rasa uses to interpret user inputs and manage conversations. The training process optimizes the model’s ability to accurately recognize intents, extract entities, and decide on appropriate actions, which is crucial for integrating seamless interactions with LanceDB and generating responses via LLMs.

In [ ]:
# Train the Rasa model
!rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/validation.py:134: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as spe

### Running Servers

The **Rasa Server** is the core component that handles user interactions. It processes incoming messages, interprets user intents, manages conversations, and generates appropriate responses based on the trained model and defined dialogue flows.

In [ ]:
import threading
import os

def run_rasa_server():
    os.system("rasa run")

threading.Thread(target=run_rasa_server).start()
# Wait for the Rasa server to start
time.sleep(35)

The **Action Server** is a separate service that executes Custom Actions defined in your project (actions.py). These actions perform specialized tasks such as querying databases, processing data, or integrating with external APIs.

In [ ]:
import threading
import os

def run_action_server():
    os.system("rasa run actions")

threading.Thread(target=run_action_server).start()
# Wait for the Rasa server to start
time.sleep(35)

### Let's test out chatbot

In [ ]:
# Function to send messages to the Rasa server
def send_message(message):
    url = "http://localhost:5005/webhooks/rest/webhook"
    payload = {
        "sender": "test_user",
        "message": message
    }
    headers = {
        "Content-Type": "application/json"
    }
    try:
        response = requests.post(url, data=json.dumps(payload), headers=headers)
        return response.json()
    except requests.exceptions.ConnectionError:
        return {"error": "Could not connect to Rasa server."}

# Example interactions
print("User: Hi")
assistant_response = send_message("Hi")
if assistant_response:
    for resp in assistant_response:
        if isinstance(resp, dict) and "text" in resp:
            print("Assistant:", resp["text"])
        else:
            print("Assistant:", resp)

print("\nUser: How do I reset my password? Explain in french")
assistant_response = send_message("How do I delete my account? Explain in french")
if assistant_response:
    print("Assistant:", end = " ")
    for resp in assistant_response:
        if isinstance(resp, dict) and "text" in resp:
            print(resp["text"])
        else:
            print(resp)

print("\nUser: Standard shipping details. Hinglish mein btao")
assistant_response = send_message("Standard shipping details. Hinglish mein btao")
if assistant_response:
    print("Assistant:", end = " ")
    for resp in assistant_response:
        if isinstance(resp, dict) and "text" in resp:
            print(resp["text"])
        else:
            print(resp)

print("\nUser: What is the weather today?")
assistant_response = send_message("What is the weather today?")
if assistant_response:
    print("Assistant:", end = " ")
    for resp in assistant_response:

        if isinstance(resp, dict) and "text" in resp:
            print(resp["text"])
        else:
            print( resp)

print("\nUser: Bye")
assistant_response = send_message("Bye")
if assistant_response:
    print("Assistant:", end = " ")
    for resp in assistant_response:
        if isinstance(resp, dict) and "text" in resp:
            print(resp["text"])
        else:
            print(resp)


User: Hi
Assistant: Hello! How can I assist you today?

User: How do I reset my password? Explain in french
Assistant: Bonjour,
Pour supprimer votre compte, veuillez contacter notre équipe d'assistance à l'adresse suivante : support@yourcompany.com. Assurez-vous d'indiquer dans l'objet de votre message "Demande de suppression de compte". Nous vous assisterons rapidement dans cette démarche.
Si vous avez d'autres questions ou si vous avez besoin d'une aide supplémentaire, n'hésitez pas à nous le faire savoir.
Cordialement,  
L'équipe de support

User: Standard shipping details. Hinglish mein btao
Assistant: Aapka swaagat hai! Humein khushi hai ki aapne humse sampark kiya. 
Hamara standard shipping policy yeh hai ki agar aapka order $75 se zyada hai, toh aapko standard shipping par koi bhi charges nahi dene honge, yani shipping free hai. Lekin agar aapka order is amount se kam hai, toh aapko standard shipping fees deni padegi.
Agar aapko aur koi sawaal hai ya madad chahiye, toh zaroor pu

### For debugging purposes

In [ ]:
!lsof -i

COMMAND    PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
node         6 root   21u  IPv6  19762      0t0  TCP *:8080 (LISTEN)
node         6 root   27u  IPv4  21268      0t0  TCP 11757d4d820b:56654->11757d4d820b:6000 (ESTABLISHED)
node         6 root   28u  IPv6  68552      0t0  TCP 11757d4d820b:8080->172.28.0.1:49662 (ESTABLISHED)
node         6 root   29u  IPv6 142453      0t0  TCP 11757d4d820b:8080->172.28.0.1:43440 (ESTABLISHED)
node         6 root   31u  IPv6  72491      0t0  TCP 11757d4d820b:8080->172.28.0.1:46072 (ESTABLISHED)
node         6 root   34u  IPv6 143833      0t0  TCP 11757d4d820b:8080->172.28.0.1:43708 (ESTABLISHED)
kernel_ma   26 root    3u  IPv4  19626      0t0  TCP 11757d4d820b:6000 (LISTEN)
kernel_ma   26 root    6u  IPv4  21269      0t0  TCP 11757d4d820b:6000->11757d4d820b:56654 (ESTABLISHED)
kernel_ma   26 root    7u  IPv4  20263      0t0  TCP 11757d4d820b:41868->11757d4d820b:9000 (ESTABLISHED)
kernel_ma   26 root    8u  IPv4  21271      0t0  TCP 11757d4d820b:

In [ ]:
!kill -9 7966

/bin/bash: line 1: kill: (7966) - No such process
